In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys

# 1) Define your working/project paths
WORK_PATH      = "./"
SOURCE_PATH    = os.path.join(WORK_PATH, "src")
HPC_PATH       = os.path.join(WORK_PATH, "hpc")

SOURCE_PATH_ABS = os.path.abspath(SOURCE_PATH)
HPC_PATH_ABS    = os.path.abspath(HPC_PATH)

# 2) Insert at front of sys.path so that imports in *this* notebook see them
#    (you can also use .append() if order doesn’t matter)
sys.path[0:0] = [SOURCE_PATH_ABS, HPC_PATH_ABS]

# 3) Export into os.environ so that any spawned subprocess or IPython magic
#    cell sees the same PYTHONPATH
old_pp = os.environ.get("PYTHONPATH", "")
paths = [SOURCE_PATH_ABS, HPC_PATH_ABS]
if old_pp:
    paths.append(old_pp)
os.environ["PYTHONPATH"] = os.pathsep.join(paths)

# --- verify ---
print("sys.path starts with:", sys.path[:2])
print("PYTHONPATH =", os.environ["PYTHONPATH"])

sys.path starts with: ['/home/pierce/Software/cmor_mdanderson/3d-1d/src', '/home/pierce/Software/cmor_mdanderson/3d-1d/hpc']
PYTHONPATH = /home/pierce/Software/cmor_mdanderson/3d-1d/src:/home/pierce/Software/cmor_mdanderson/3d-1d/hpc


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
import continuation

def run_example_sweep_and_plot():
    """
    Runs an example sweep using run_sweep and plots the results.
    """
    # Define sweep parameters
    sweep_name = "gamma_a"
    sweep_values = np.logspace(1e-5, 1e-3, 1).tolist()
    target_flow = 5.0e-6  # Target total_flow
    default_params = [1e-4, 1e-4, 1e-4] # [gamma, gamma_a, gamma_R]
    max_iterations = 50
    results_directory = "sweep_results_example"

    print(f"Running sweep for '{sweep_name}' with values from {sweep_values[0]:.2e} to {sweep_values[-1]:.2e}...")

    # Ensure the results directory exists and is clean for this example
    output_dir_path = Path(results_directory)
    if not output_dir_path.exists():
        output_dir_path.mkdir(parents=True)
    
    # Run the sweep
    continuation.run_sweep(
        sweep_name=sweep_name,
        sweep_values=sweep_values,
        target=target_flow,
        x_default=default_params,
        maxiter=max_iterations,
        directory=results_directory,
    )

    # --- Plotting the results ---
    csv_path = os.path.join(results_directory, "sweep_results.csv")
    if Path(csv_path).exists():
        df = pd.read_csv(csv_path)

        print("\nSweep results (first 5 rows):")
        print(df.head())

        plt.figure(figsize=(10, 6))
        plt.plot(df[sweep_name], df["total_flow"], marker='o', linestyle='-')
        plt.axhline(y=target_flow, color='r', linestyle='--', label=f'Target Flow ({target_flow:.1e} m³/s)')
        plt.title(f'Total Flow vs. {sweep_name} (Target: {target_flow:.1e} m³/s)')
        plt.xlabel(f'{sweep_name} (Pa·s⁻¹·m⁻²)')
        plt.ylabel('Total Flow (m³/s)')
        plt.grid(True)
        plt.legend()
        plt.xscale('log') # Use log scale for gamma_a as values can vary widely
        plt.tight_layout()
        plot_filename = Path(results_directory) / f"flow_vs_{sweep_name}_plot.png"
        plt.savefig(plot_filename)
        print(f"\nPlot saved to: {plot_filename}")
        plt.show()
    else:
        print(f"Error: Results CSV not found at {csv_path}")

if __name__ == "__main__":
    run_example_sweep_and_plot()

/home/pierce/Software/cmor_mdanderson/lib/python3.13/site-packages/block/__init__.py:15: UserWarning: The cbc.block repository has moved to https://github.com/blocknics/cbc.block
  warnings.warn('The cbc.block repository has moved to https://github.com/blocknics/cbc.block', UserWarning)


Missing HsMG for fract norm computing
Running sweep for 'gamma_a' with values from 1.00e+00 to 1.00e+00...

step 0: set gamma_a = 1.000e+00


Averaging over 224 cells: 100%|█████████████| 224/224 [00:00<00:00, 2145.24it/s]
